In [1]:
import os
import csv
import spacy
from tqdm import tqdm

# Khởi tạo NLP parser
nlp = spacy.load("en_core_web_sm")

# Thư mục chứa caption (Sentences/)
sentences_dir = "E:/Download/Flickr30k/Sentences/"

# Danh sách triplet sẽ lưu
triplets = []

# Hàm đơn giản trích (subject, predicate, object)
def extract_triplet(text):
    doc = nlp(text)
    for sent in doc.sents:
        for token in sent:
            if token.pos_ == "VERB":
                subj = [w for w in token.lefts if w.dep_ in ["nsubj", "nsubjpass"]]
                obj = [w for w in token.rights if w.dep_ in ["dobj", "pobj"]]
                if subj and obj:
                    return (subj[0].text.lower(), token.lemma_.lower(), obj[0].text.lower())
    return None

# Duyệt qua từng file caption
file_names = [f for f in os.listdir(sentences_dir) if f.endswith(".txt")]

for file_name in tqdm(file_names):
    image_id = file_name.replace(".jpg.txt", "")  # tên ảnh
    with open(os.path.join(sentences_dir, file_name), "r", encoding="utf-8") as f:
        lines = f.readlines()
        for line in lines:
            line = line.strip()
            triplet = extract_triplet(line)
            if triplet:
                triplets.append((*triplet, image_id))

# Ghi ra file CSV
with open("flickr_triplets.csv", "w", newline='', encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["subject", "predicate", "object", "image_id"])
    writer.writerows(triplets)

print(f"✅ Đã trích xuất {len(triplets)} triplet từ folder 'Sentences/' vào flickr_triplets.csv")


100%|████████████████████████████████████████████████████████████████████████████| 31783/31783 [36:15<00:00, 14.61it/s]

✅ Đã trích xuất 47147 triplet từ folder 'Sentences/' vào flickr_triplets.csv
